# MNIST E2E on Kubeflow

* **This is a work in progress; this is not read for users yet**

This example guides you through the process of taking an example model, modifying it to run better within Kubeflow, and serving the resulting trained model.

## Prepare model

There is a delta between existing distributed mnist examples and what's needed to run well as a TFJob.

Basically, we must:

1. Add options in order to make the model configurable.
1. Use `tf.estimator.train_and_evaluate` to enable model exporting and serving.
1. Define serving signatures for model serving.

The resulting model is [model.py](model.py).

### Verify we have a GCP account

* The cell below checks that this notebook was spawned with credentials to access GCP


In [1]:
import os
from oauth2client.client import GoogleCredentials
credentials = GoogleCredentials.get_application_default()

## Install Required Libraries

Import the libraries required to train this model.

In [3]:
import notebook_setup
notebook_setup.notebook_setup()

pip installing requirements.txt


CalledProcessError: Command '['pip3', 'install', '--user', '-r', 'requirements.txt']' returned non-zero exit status 1.

### Configure The Docker Registry For Kubeflow Fairing

* In order to build docker images from your notebook we need a docker registry where the images will be stored
* Below you set some variables specifying a [GCR container registry](https://cloud.google.com/container-registry/docs/)
* Kubeflow Fairing provides a utility function to guess the name of your GCP project

In [5]:
# Setting up google container repositories (GCR) for storing output containers
# You can use any docker container registry istead of GCR
GCP_PROJECT = fairing.cloud.gcp.guess_project_name()
DOCKER_REGISTRY = 'gcr.io/{}/fairing-job'.format(GCP_PROJECT)

NameError: name 'fairing' is not defined

In [ ]:
## Use Kubeflow fairing to build the docker image

* You will use kubeflow fairing's kaniko builder to build a docker image that includes all your dependencies
  * You use kaniko because you want to be able to run `pip` to install dependencies
  * Kaniko gives you the flexibility to build images from Dockerfiles

In [ ]:
# TODO(https://github.com/kubeflow/fairing/issues/426): We should get rid of this once the default 
# Kaniko image is updated to a newer image than 0.7.0.
from kubeflow.fairing import constants
constants.constants.KANIKO_IMAGE = "gcr.io/kaniko-project/executor:v0.14.0"

In [ ]:
raise ValueError("Need to update this don't want to use the notebook preprocessor")
from kubeflow.fairing.builders import cluster

# output_map is a map of extra files to add to the notebook.
# It is a map from source location to the location inside the context.
output_map =  {
    "Dockerfile": "Dockerfile",
    "requirements.txt": "requirements.txt",
}


preprocessor = ConvertNotebookPreprocessorWithFire(class_name='ModelServe', notebook_file='build-train-deploy.ipynb',
                                                   output_map=output_map)

if not preprocessor.input_files:
    preprocessor.input_files = set()
input_files=["xgboost_util.py", "mockup-model.dat"]
preprocessor.input_files =  set([os.path.normpath(f) for f in input_files])
preprocessor.preprocess()

In [ ]:
raise ValueError("Update the base image for TF")
base_image = "gcr.io/kubeflow-images-public/tensorflow-1.14.0-notebook-cpu:v0.7.0"
# We use a custom Dockerfile 
cluster_builder = cluster.cluster.ClusterBuilder(registry=DOCKER_REGISTRY,
                                                 base_image=base_image,
                                                 preprocessor=preprocessor,
                                                 dockerfile_path="Dockerfile",
                                                 pod_spec_mutators=[fairing.cloud.gcp.add_gcp_credentials_if_exists],
                                                 context_source=cluster.gcs_context.GCSContextSource())
cluster_builder.build()

## Build and push image.

With our code ready, we will now build/push the docker image, or use the existing image `gcr.io/kubeflow-ci/mnist/model:latest` without building and pushing.

```
DOCKER_URL=docker.io/reponame/mytfmodel:tag # Put your docker registry here
docker build . --no-cache  -f Dockerfile.model -t ${DOCKER_URL}

docker push ${DOCKER_URL}
```